In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
geolocator = Nominatim(user_agent="geo_goiania")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [3]:
LAT_MIN, LAT_MAX = -16.828, -16.546
LON_MIN, LON_MAX = -49.372, -49.043
CENTRAL_COORDS = (-16.6869, -49.2648)

In [4]:
df = pd.read_excel(
    r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos Processados\BOMBEIROS_TRATADO.xlsx")

In [5]:
def geocode_goiania(address_line):
    # Normaliza e separa partes do endereço por - ou /
    parts = [p.strip() for p in address_line.replace('/', '-').split('-') if p.strip()]
    
    # Tenta geocodificar do mais completo para o mais genérico
    for i in range(len(parts), 0, -1):
        partial_address = ", ".join(parts[:i]) + ", Goiânia, GO, Brasil"
        location = geocode(partial_address)
        if location:
            lat, lon = location.latitude, location.longitude
            # Checa se está dentro do bounding box
            if LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX:
                return pd.Series([lat, lon])
            else:
                # Fora do bounding box, retorna coordenada central
                return pd.Series(CENTRAL_COORDS)
    
    # Se todas as tentativas falharem, retorna coordenada central
    return pd.Series(CENTRAL_COORDS)

In [14]:
with open("pontos_DC.txt", "r", encoding="latin1") as f:
    addresses = [line.strip() for line in f if line.strip()]

In [7]:
df = pd.DataFrame(addresses, columns=['ENDEREÇO'])

In [8]:
df[['LATITUDE', 'LONGITUDE']] = df['ENDEREÇO'].apply(geocode_goiania)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Av. T, Goiânia, GO, Brasil',), **{}).
Traceback (most recent call last):
  File "c:\Users\thiag\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
  File "c:\Users\thiag\anaconda3\Lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
  File "c:\Users\thiag\anaconda3\Lib\http\client.py", line 1430, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "c:\Users\thiag\anaconda3\Lib\http\client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^^
  File "c:\Users\thiag\anaconda3\Lib\http\client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^
  File "c:\Users\thiag\anaconda3\Lib\socket.py", line 719, in readinto
    return self._sock.

In [13]:
df.to_csv("pontos_dc_geocodificados.csv", index=False, encoding="utf-8")

# Validar

In [21]:
import folium
from folium.plugins import HeatMap

In [22]:
df_risco = pd.read_csv("pontos_dc_geocodificados.csv")  # azul
excel_path = r"C:\Users\thiag\OneDrive\Área de Trabalho\TCC 2\Scripts\Arquivos para BI\DADOS_GEOCODED_UNIFICADOS.xlsx"
df_ocorrencias = pd.read_excel(excel_path)  # vermelho

In [23]:
mapa = folium.Map(location=[-16.6869, -49.2648], zoom_start=12)  

In [24]:
# Pontos de risco (azul)
for _, row in df_risco.iterrows():
    if pd.notna(row['LATITUDE']) and pd.notna(row['LONGITUDE']):
        folium.CircleMarker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            color='blue',
            fill=True,
            fill_opacity=0.7,
            popup=row['ENDEREÇO']
        ).add_to(mapa)

# Pontos de alagamento (vermelho)
for _, row in df_ocorrencias.iterrows():
    if pd.notna(row['LATITUDE']) and pd.notna(row['LONGITUDE']):
        folium.CircleMarker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            color='red',
            fill=True,
            fill_opacity=0.7,
            popup=row['LOCAL']
        ).add_to(mapa)

In [26]:
#heat_data = [[row['LATITUDE'], row['LONGITUDE']] for _, row in df_ocorrencias.iterrows() if pd.notna(row['LATITUDE'])]

heat_data = [[row['LATITUDE'], row['LONGITUDE']] 
            for _, row in df_ocorrencias.iterrows() if pd.notna(row['LATITUDE'])]


#HeatMap(heat_data, radius=25, blur=15, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 0.8: 'red'}).add_to(mapa)

HeatMap(
    heat_data,
    radius=15,      
    blur=10,        
    max_zoom=13,    
    gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 0.8: 'red'}  # define cores
).add_to(mapa)

mapa.save("mapa_risco_alagamentos.html")
